<a href="https://colab.research.google.com/github/ttogle918/classify_instrument_by_CNN/blob/master/wav_to_csv_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 파일 내용
+ 01.
  + audio_df_01_spt.pickle : spt(np.array함)에 대한 list
  + audio_df_01_ins.pickle : ins에 대한 list
  + audio_df_01_note_list.pickle : note_list에 대한 list

+ 02
  + audio_df_02_spt.pickle : spt에 대한 list
  + audio_df_02_ins.pickle : ins에 대한 list
  + audio_df_02_note_list.pickle : note_list에 대한 list
  + audio_df_02_df.pickle : spt(not array화), ins, note에 대한 dataframe


#### wav파일을 주파수 대역으로 변환하기 위해 필요한 패키지 설치

In [1]:
!pip install librosa

#### google drive와 연결

output.wav는 2초 간격으로 128종 악기별로 50개의 음이 저장되어 있는 파일로 46개의 타악기 소리가 들어가 있다.

다운로드 링크 : https://drive.google.com/file/d/1my0VG8XBD6nJQkRj30m-1zHUaMk4IXml/view



In [2]:
from google.colab import drive

drive.mount('/content/drive')
path = '/content/drive/My Drive/Colab Notebooks/code_states/s4_project'

Mounted at /content/drive


필요한 패키지 선언

In [3]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import itertools
import pandas as pd

### wav -> 주파수 대역

wav는 매 순간의 음압을 측정하여 그 수치를 저장한 형태입니다. <br/>하지만 우리는 음악의 높이와 세기를 듣기 때문에 wav파일을 음악 분석에 사용하기에 적절하지 않습니다.<br/>
따라서 변환이 필요합니다.

#### 변환 방법에는 Constant-Q 변환을 사용할 것입니다.

Constant-Q 변환은 주파수 축이 로그 단위로 변환되고<br/>
주파수에 따라 해상도가 다양하게 처리되기 때문에(저주파 - 저해상도, 고주파-고해상도) <br/>
음악을 처리하는 데에 푸리에 변환보다 유리합니다.

- spt : 스펙토그램
- ins : 악기 번호
- note : 음 높이

In [4]:
spt = []
ins = []
note_list = []
n = 0

for instrument, note in itertools.product(range(128), range(50)):
    y, sr = librosa.load(path + '/output.wav', sr=None, offset=n, duration=2.0) # n초지점부터 2초까지만 데이터를 읽어옵니다.
    n += 2
    # 데이터를 늘리기 위해 white 노이즈를 섞은 버전도 함께 변환합니다
    # 시간 대역 데이터를 옥타브당 24단계로, 총 7옥타브로 변환할 겁니다.
    
    # 0 : 원본 데이터, 1e-4 : 노이즈를 0.0001만큼 넣은 것, 1e-3은 노이즈를 0.001만큼 넣은 것
    # 데이터는 원본의 3배가 된다.
    for r in (0, 1e-4, 1e-3): # 1e-4 == 0.0001, 1e-3 == 0.001
        # librosa.cqt : audio 신호를 constant-Q 로 변환시키는 메소드
        # y : audio time series
        # sr : y의 샘플링 비율
        # hop_length : 연속적인 cqt 컬럼 사이의 샘플의 수
        # fmin에서 시작한 빈도 bin의 수
        # 옥타브마다 bins의 수
        ret = librosa.cqt(y + ((np.random.rand(*y.shape) - 0.5) * r if r else 0), sr, hop_length=1024, n_bins=24*7, bins_per_octave=24)
        # 주파수의 위상은 관심없고, 세기만 보겠으니 절대값을 취해줍니다
        ret = np.abs(ret)
        spt.append(ret)
        ins.append(instrument)
        note_list.append(38 + note)

In [5]:
df = pd.DataFrame({'spt': spt, 'ins':ins, 'note':note})

In [8]:
import pickle
# checkpoint 생성 ( data 저장 )
with open(path + '/audio_df_01_spt.pickle', 'wb') as f:
    pickle.dump(spt, f)
with open(path + '/audio_df_01_ins.pickle', 'wb') as f:
    pickle.dump(ins, f)

with open(path + '/audio_df_01_note_list.pickle', 'wb') as f:
    pickle.dump(note_list, f)
with open(path + '/audio_df_01.pickle', 'wb') as f:
    pickle.dump(df, f)

In [9]:
# checkpoint 불러오기 ( 저장된 파일 확인 )
with open(path + '/audio_df_01.pickle', 'rb') as f:
    pickled = pickle.load(f)

print(pickled.shape)
pickled.tail()

(19200, 3)


,spt,ins,note
19195,"[[0.0012306672757767421, 0.001225163936175184,...",127,49
19196,"[[0.0018502094028348146, 0.0018442328599959919...",127,49
19197,"[[0.0013353681, 0.0013276788, 0.0013065297, 0....",127,49
19198,"[[0.0012678198658570064, 0.0012603254439365488...",127,49
19199,"[[0.0007400535988885903, 0.0007348174757863667...",127,49


In [10]:
df.tail()

,spt,ins,note
19195,"[[0.0012306672757767421, 0.001225163936175184,...",127,49
19196,"[[0.0018502094028348146, 0.0018442328599959919...",127,49
19197,"[[0.0013353681, 0.0013276788, 0.0013065297, 0....",127,49
19198,"[[0.0012678198658570064, 0.0012603254439365488...",127,49
19199,"[[0.0007400535988885903, 0.0007348174757863667...",127,49


In [ ]:
spt = []
ins = []
note_list = []

for note in range(46):
    y, sr = librosa.load(path + '/output.wav', sr=None, offset=n, duration=2.0)
    n += 2
 
    for r, s in itertools.product([0, 1e-5, 1e-4, 1e-3], range(7)):

      ret = librosa.cqt(y + ((np.random.rand(*y.shape) - 0.5) * r * s if r else 0), sr, hop_length=1024, n_bins=24 * 7, bins_per_octave=24)
      ret = np.abs(ret)
      spt.append(ret)
      ins.append(note+128)
      note_list.append(0)
    # 아래의 코드는 변환된 주파수 대역의 스펙토그램을 보여줍니다.
    # librosa.display.specshow(librosa.amplitude_to_db(np.abs(ret), ref=np.max), sr=sr, x_axis='time', y_axis='cqt_note')
    # plt.colorbar(format='%+2.0f dB')
    # plt.title('Constant-Q power spectrum')
    # plt.tight_layout()
    # plt.show()

In [ ]:
df = pd.DataFrame({'spt': spt, 'ins': ins, 'note': note_list })
print(df.shape)
df.tail()

(1288, 3)


,spt,ins,note
1283,"[[0.003795179078229729, 0.003774560596158327, ...",173,0
1284,"[[0.004901618141738685, 0.004878211192395115, ...",173,0
1285,"[[0.0040358022231501344, 0.003999411365052383,...",173,0
1286,"[[0.004431604010000036, 0.004415313115321793, ...",173,0
1287,"[[0.003406624151416973, 0.0033999772982200266,...",173,0


In [ ]:
import pickle
# checkpoint 생성 ( data 저장 )
with open(path + '/audio_df_02_spt.pickle', 'wb') as f:
    pickle.dump(spt, f)
with open(path + '/audio_df_02_ins.pickle', 'wb') as f:
    pickle.dump(ins, f)

with open(path + '/audio_df_02_note_list.pickle', 'wb') as f:
    pickle.dump(note_list, f)

with open(path + '/audio_df_02.pickle', 'wb') as f:
    pickle.dump(df, f)

In [ ]:
import pickle

with open(path + '/audio_df_01_spt.pickle', 'rb') as f:
    spt = pickle.load(f)
with open(path + '/audio_df_01_ins.pickle', 'rb') as f:
    ins = pickle.load(f)
with open(path + '/audio_df_01_note_list.pickle', 'rb') as f:
    note_list = pickle.load(f)